# Рекомендация тарифов

Предоставлены данные о поведении клиентов, которые уже перешли на эти тарифы.
Нужно построить модель для задачи классификации, которая выберет подходящий тариф. 

Метрика качества - *accuracy*.

## 1. Изучение общей информации о данных

In [1]:
# Все библиотеки, необходимые для выполнения проекта 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore') 

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
# Посмотрим на датафрейм изнутри
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### Вывод

Задача была упрощена тем, что предобработка не нужна: нужно изучить данные. Таблица состоит из данных, которые характеризуют поведение каждого пользователя (всего их 3214) в конкретный месяц. Тип данных приведен к нужным типам, однако возможно использование целочисленного типа (int) для столбцов с количество сообщений и звонков. 

## 2. Разбиение данных на тренировочную, валидационную и тестовую выборки

In [7]:
# Разобьем исходные данные на обучающую (df_train), валидационную (df_val) и тестовую выборки (df_test) в соотношении 3:1:1
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)
df_train, df_val = train_test_split(df_train, test_size=0.25, random_state=12345)
# Проверим получилось ли разделить данные в необходимом соотношении
df_train.shape[0], df_test.shape[0], df_val.shape[0]

(1928, 643, 643)

In [8]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_val = df_val.drop(['is_ultra'], axis=1)
target_val = df_val['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
features_test.shape, features_val.shape,

((643, 4), (643, 4))

In [9]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

## 3. Исследование моделей

Исследование качества разных моделей с разными гиперпараметрами.

Первый тип модели - решающее дерево.

In [10]:
# Проверим влияние различных гиперпараметров на качество модели решающего дерева
dt = DecisionTreeClassifier(random_state = 12345)
parameters_dt = {'criterion': ['gini', 'entropy'], # критерий
                 'max_depth':range(1, 50), # глубина дерева
                 'min_samples_split': [0.1, 0.5, 0.7, 1.0], # минимальное количество выборок для разделения
                 'min_samples_leaf': range(1, 20), # минимальное количесто объектов в листе
                 'max_features':['auto', 'log2', 'sqrt', None]} # максимальное количество фич
search_dt = RandomizedSearchCV(dt, parameters_dt, cv=5, n_jobs = -1)
search_dt.fit(features_train, target_train)
best_dt = search_dt.best_estimator_
predict_dt = best_dt.predict(features_val)
print('Accuracy для Decision Tree на тренировочной выборке:', best_dt.score(features_train, target_train))
print('Accuracy для Decision Tree на валидационной выборке:', best_dt.score(features_val, target_val))

Accuracy для Decision Tree на тренировочной выборке: 0.8174273858921162
Accuracy для Decision Tree на валидационной выборке: 0.7713841368584758


Вторая модель - случайный лес.


In [11]:
# Проверим влияние различных гиперпараметров на качество модели случайного леса
rf = RandomForestClassifier(random_state = 12345)
parameters_rf = {'n_estimators': range(1, 200, 10), # количество деревьев 
                 'criterion': ['gini', 'entropy'], # критерий
                 'max_depth':range(1, 50, 1), # глубина дерева
                 'min_samples_split': [0.1, 0.5, 0.7, 1.0], # минимальное количество выборок для разделения
                 'min_samples_leaf': range(1, 10), # минимальное количесто объектов в листе
                 'max_features':['auto', 'log2', 'sqrt', None]} # максимальное количество фич
search_rf = RandomizedSearchCV(rf, parameters_rf, cv=5, n_jobs = -1)
search_rf.fit(features_train, target_train)
best_rf = search_rf.best_estimator_
predict_dt = best_rf.predict(features_val)
print('Accuracy для Random Forest на тренировочной выборке:', best_rf.score(features_train, target_train))
print('Accuracy для Random Forest на валидационной выборке:', best_rf.score(features_val, target_val))

Accuracy для Random Forest на тренировочной выборке: 0.8184647302904564
Accuracy для Random Forest на валидационной выборке: 0.776049766718507


Третья модель - логистическая регрессия.

In [12]:
# Проверим влияние "параметров" на качество модели логистической регрессии
lr = LogisticRegression(random_state = 12345)
parameters_lr_1 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l1'], # Регуляризация 
                 'solver':['liblinear', 'saga']} # функция потерь
parameters_lr_2 = {'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 5, 10, 20, 50], # обратная сила регуляризации
                 'penalty':['l2'], # Регуляризация 
                 'solver':['lbfgs', 'saga', 'sag', 'newton-cg']} # функция потерь

search_lr_1 = RandomizedSearchCV(lr, parameters_lr_1, cv=5, n_jobs = -1)
search_lr_1.fit(features_train, target_train)
best_lr_1 = search_lr_1.best_estimator_
predict_lr_1 = best_lr_1.predict(features_val)

search_lr_2 = RandomizedSearchCV(lr, parameters_lr_2, cv=5, n_jobs = -1)
search_lr_2.fit(features_train, target_train)
best_lr_2 = search_lr_2.best_estimator_
predict_lr_2 = best_lr_2.predict(features_val)

print('Accuracy для Logistic Regression (регуляризация l1) на тренировочной выборке :', best_lr_1.score(features_train, target_train))
print('Accuracy для Logistic Regression (регуляризация l1) на валидационной выборке:', best_lr_1.score(features_val, target_val))
print()
print('Accuracy для Logistic Regression (регуляризация l2) на тренировочной выборке:', best_lr_2.score(features_train, target_train))
print('Accuracy для Logistic Regression (регуляризация l2) на валидационной выборке:', best_lr_2.score(features_val, target_val))

Accuracy для Logistic Regression (регуляризация l1) на тренировочной выборке : 0.7515560165975104
Accuracy для Logistic Regression (регуляризация l1) на валидационной выборке: 0.7278382581648523

Accuracy для Logistic Regression (регуляризация l2) на тренировочной выборке: 0.7510373443983402
Accuracy для Logistic Regression (регуляризация l2) на валидационной выборке: 0.7262830482115086


In [13]:
data_for_score_val = [['decision tree', best_dt.score(features_val, target_val)],
                      ['random forest', best_rf.score(features_val, target_val)],
                      ['logistic regression (l1)', best_lr_1.score(features_val, target_val)], 
                      ['logistic regression (l2)', best_lr_2.score(features_val, target_val)]]
columns_for_score_val = ['model', 'accuracy']
table_for_score_val = pd.DataFrame(data = data_for_score_val, columns = columns_for_score_val)
table_for_score_val

,model,accuracy
0,decision tree,0.771384
1,random forest,0.776050
2,logistic regression (l1),0.727838
3,logistic regression (l2),0.726283


### Вывод

   <ul>Были исследованы три модели: как видно, наибольшей точностью модели обладает модель "случайного леса", что вполне логично (много деревьев по сравнению с одним решающим деревом); однако "решающее дерево" имеет близкое значение по точности к "случайному лесу", и даже "обгоняет" логистическую регрессию - возможно, модель одного дерева переобучена, либо параметры для решающего дерева были подобраны намного лучше, чем для остальных моделей. </ul>
   <ul> Если для работы необходимо не только качество, но и высокоя скорость работы и обучения, то предпочтительнее всего использовать модель решающего дерева - качество выше, чем у логистической регрессии, и близкое к "случайному лесу". </ul>

## 4.Проверка качества модели на тестовой выборке.

In [14]:
# Так как наибольшей точностью все-таки обладает модель случайного леса, то проанализируем её на тестовой выборке
accuracy_rf_test = best_rf.score(features_test, target_test)
print('Accuracy для Random Forest на тестовой выборке:', accuracy_rf_test)

Accuracy для Random Forest на тестовой выборке: 0.7947122861586314


### Вывод

Качество модели случайного леса на тестовой выборке оказалось даже выше, чем необходимо 

## 5. Проверка модели на адекватность

In [15]:
# Посмотрим на соотношение классов в датасете 
df_smart = df[df['is_ultra'] == 0]
df_ultra = df[df['is_ultra'] == 1]
df_smart_len = df_smart.shape[0] 
df_ultra_len = df_ultra.shape[0]
ratio = df_smart_len / (df_smart_len + df_ultra_len)
print('Соотношение двух классов (тарифов "ультра" и "смарт"): ', ratio)
# Считается, что модель считается адекватной, если её точность выше, чем доля бОльшего класса в выборке,
# поэтому можно считать, что все модели, рассмотренные в шаге 4, могут быть адекватными

Соотношение двух классов (тарифов "ультра" и "смарт"):  0.693528313627878


Сравним настроенные модели с базовыми:

Первая модель - решающее дерево.

In [16]:
dt_base = DecisionTreeClassifier(random_state=12345)
dt_base.fit(features_train, target_train)
accuracy_dt_base_test = dt_base.score(features_test, target_test)
print('Точность модели "решающее дерево":', accuracy_dt_base_test)

Точность модели "решающее дерево": 0.7309486780715396


Вторая модель - случайный лес.

In [17]:
rf_base = RandomForestClassifier(random_state=12345)
rf_base.fit(features_train, target_train)
accuracy_rf_base_test =  rf_base.score(features_test, target_test)
print('Точность модели "случайного леса":', accuracy_rf_base_test)

Точность модели "случайного леса": 0.7869362363919129


Третья модель - логистическая регрессия.

In [18]:
lr_base = LogisticRegression(random_state=12345)
lr_base.fit(features_train, target_train)
accuracy_lr_base_test = lr_base.score(features_test, target_test)
print('Точность модели "логичестической регрессии":', accuracy_lr_base_test)

Точность модели "логичестической регрессии": 0.702954898911353


In [19]:
data_base_create_ml = [['decision_tree', accuracy_dt_base_test, best_dt.score(features_test, target_test)], 
                    ['random_forest', accuracy_rf_base_test, best_rf.score(features_test, target_test)], 
                    ['logistic_regression', accuracy_lr_base_test, best_lr_1.score(features_test, target_test)]]
columns_base_create_ml = ['model', 'base','create']
table_base_create_ml = pd.DataFrame(data = data_base_create_ml, columns = columns_base_create_ml)
table_base_create_ml

,model,base,create
0,decision_tree,0.730949,0.793157
1,random_forest,0.786936,0.794712
2,logistic_regression,0.702955,0.758942


### Вывод

<ul> Были проанализированы модели решающего дерева, случайного леса и логистической регрессии на адекватность: получено, что все модели адекватны и имеют место для использования в данной задаче. </ul>
<ul>Примечательно то, что все три модели обладают accuracy > 0.75, и, таким образом, поставленная задача была выполнена. В дальнейшем нужно принять решение о выборе быстродействия модели - нужна ли в этой задачи максимальная точность, но при этом низкое быстродействие, или же наоборот - можно пренебречь несколькими процентами точности, но при этом модель будет гораздо быстрее.</ul>